
## **Name: Noor Ul Ain Khurshid**




There are several forms of LSTM that can be used. At the very basic level a variation of using LSTM would be stack LSTM layers together. A sample code is given below on how it can be easily done using keras.

Now adapting either from  jupyter notebook 7b or 7c to stack 2, 3 or more layers togethers. Using the same datasets given in the notebook. Including scripts here with observations


In [3]:
import os
import re
import tarfile

import requests
!pip install pugnlp
from pugnlp.futil import path_status, find_files

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.10/dist-packages/pugnlp/constants.py:137: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  DATE_TYPES = (datetime.datetime, datetime.date)
/usr/local/lib/python3.10/dist-packages/pugnlp/constants.py:159: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  ZERO_TIMESTAMP = pd.Timestamp('1970-01-01 00:00:00', tz='utc')


In [5]:
# From the nlpia package for downloading data too big for the repo
import tqdm
BIG_URLS = {
    'w2v': (
        'https://www.dropbox.com/s/965dir4dje0hfi4/GoogleNews-vectors-negative300.bin.gz?dl=1',
        1647046227,
    ),
    'slang': (
        'https://www.dropbox.com/s/43c22018fbfzypd/slang.csv.gz?dl=1',
        117633024,
    ),
    'tweets': (
        'https://www.dropbox.com/s/5gpb43c494mc8p0/tweets.csv.gz?dl=1',
        311725313,
    ),
    'lsa_tweets': (
        'https://www.dropbox.com/s/rpjt0d060t4n1mr/lsa_tweets_5589798_2003588x200.tar.gz?dl=1',
        3112841563,  # 3112841312,
    ),
    'imdb': (
        'https://www.dropbox.com/s/yviic64qv84x73j/aclImdb_v1.tar.gz?dl=1',
        3112841563,  # 3112841312,
    ),
}

In [6]:
# These functions are part of the nlpia package which can be pip installed and run from there.
def dropbox_basename(url):
    filename = os.path.basename(url)
    match = re.findall(r'\?dl=[0-9]$', filename)
    if match:
        return filename[:-len(match[0])]
    return filename

def download_file(url, data_path='.', filename=None, size=None, chunk_size=4096, verbose=True):
    """Uses stream=True and a reasonable chunk size to be able to download large (GB) files over https"""
    if filename is None:
        filename = dropbox_basename(url)
    file_path = os.path.join(data_path, filename)
    if url.endswith('?dl=0'):
        url = url[:-1] + '1'  # noninteractive download
    if verbose:
        tqdm_prog = tqdm
        print('requesting URL: {}'.format(url))
    else:
        tqdm_prog = no_tqdm
    r = requests.get(url, stream=True, allow_redirects=True)
    size = r.headers.get('Content-Length', None) if size is None else size
    print('remote size: {}'.format(size))

    stat = path_status(file_path)
    print('local size: {}'.format(stat.get('size', None)))
    if stat['type'] == 'file' and stat['size'] == size:  # TODO: check md5 or get the right size of remote file
        r.close()
        return file_path

    print('Downloading to {}'.format(file_path))

    with open(file_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=chunk_size):
            if chunk:  # filter out keep-alive chunks
                f.write(chunk)

    r.close()
    return file_path

def untar(fname):
    if fname.endswith("tar.gz"):
        with tarfile.open(fname) as tf:
            tf.extractall()
    else:
        print("Not a tar.gz file: {}".format(fname))

In [7]:
#  UNCOMMENT these 2 lines if you haven't already download the word2vec model and the imdb dataset
download_file(BIG_URLS['w2v'][0])
untar(download_file(BIG_URLS['imdb'][0]))

requesting URL: https://www.dropbox.com/s/965dir4dje0hfi4/GoogleNews-vectors-negative300.bin.gz?dl=1
remote size: 1647046227
local size: None
requesting URL: https://www.dropbox.com/s/yviic64qv84x73j/aclImdb_v1.tar.gz?dl=1
remote size: 84125825
local size: None


But what is the memory? The memory is going to be represented by a vector that is the
same number of elements as neurons in the cell. Your example has a relatively simple
50 neurons, so the memory unit will be a vector of floats that is 50 elements long

In [8]:
import glob
import os

from random import shuffle

def pre_process_data(filepath):
    """
    This is dependent on your training data source but we will try to generalize it as best as possible.
    """
    positive_path = os.path.join(filepath, 'pos')
    negative_path = os.path.join(filepath, 'neg')
    
    pos_label = 1
    neg_label = 0
    
    dataset = []
    
    for filename in glob.glob(os.path.join(positive_path, '*.txt')):
        with open(filename, 'r') as f:
            dataset.append((pos_label, f.read()))
            
    for filename in glob.glob(os.path.join(negative_path, '*.txt')):
        with open(filename, 'r') as f:
            dataset.append((neg_label, f.read()))
    
    shuffle(dataset)
    
    return dataset

In [9]:
from nltk.tokenize import TreebankWordTokenizer
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True, limit=200000)

def tokenize_and_vectorize(dataset):
    tokenizer = TreebankWordTokenizer()
    vectorized_data = []
    expected = []
    for sample in dataset:
        tokens = tokenizer.tokenize(sample[1])
        sample_vecs = []
        for token in tokens:
            try:
                sample_vecs.append(word_vectors[token])

            except KeyError:
                pass  # No matching token in the Google w2v vocab
            
        vectorized_data.append(sample_vecs)

    return vectorized_data

In [10]:
def collect_expected(dataset):
    """ Peel of the target values from the dataset """
    expected = []
    for sample in dataset:
        expected.append(sample[0])
    return expected

In [11]:
def pad_trunc(data, maxlen):
    """ For a given dataset pad with zero vectors or truncate to maxlen """
    new_data = []

    # Create a vector of 0's the length of our word vectors
    zero_vector = []
    for _ in range(len(data[0][0])):
        zero_vector.append(0.0)

    for sample in data:
 
        if len(sample) > maxlen:
            temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)
        else:
            temp = sample
        new_data.append(temp)
    return new_data

In [12]:
import numpy as np

dataset = pre_process_data('./aclImdb/train')
vectorized_data = tokenize_and_vectorize(dataset)
expected = collect_expected(dataset)

split_point = int(len(vectorized_data)*.2)
split_point2 = int(len(vectorized_data)*.25)

x_train = vectorized_data[:split_point]
y_train = expected[:split_point]
x_test = vectorized_data[split_point:split_point2]
y_test = expected[split_point:split_point2]

maxlen = 400
batch_size = 32         # How many samples to show the net before backpropogating the error and updating the weights
embedding_dims = 300    # Length of the token vectors we will create for passing into the Convnet
epochs = 20

x_train = pad_trunc(x_train, maxlen)
x_test = pad_trunc(x_test, maxlen)

x_train = np.reshape(x_train, (len(x_train), maxlen, embedding_dims))
y_train = np.array(y_train)
x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims))
y_test = np.array(y_test)

# **Building LSTM Network**

One import and one line of Keras code changed. But a great deal more is going on
under the surface. From the summary, you can see you have many more parameters to
train than you did in the SimpleRNN from last chapter for the same number of neurons
(50). Recall the simple RNN had the following weights:

*   300 (one for each element of the input vector)
*   1 (one for the bias term)
*   50 (one for each neuron’s output from the previous time step)

For a total of 351 per neuron.

351 * 50 = 17,550 for the layer

The cells have three gates (a total of four neurons):

17,550 * 4 = 70,200

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM

num_neurons = 50

print('Build model...')
model = Sequential()

model.add(LSTM(num_neurons, return_sequences=True, input_shape=(maxlen, embedding_dims)))
model.add(LSTM(num_neurons, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(num_neurons))
model.add(Dropout(.2))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile('rmsprop', 'binary_crossentropy',  metrics=['accuracy'])
print(model.summary())

Build model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 400, 50)           70200     
                                                                 
 lstm_1 (LSTM)               (None, 400, 50)           20200     
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 flatten (Flatten)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 110,651
Trainable params: 1

# **Training LSTM Network**

In [14]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))
model_structure = model.to_json()
with open("lstm_model1.json", "w") as json_file:
    json_file.write(model_structure)

model.save_weights("lstm_weights1.h5")
print('Model saved.')

Epoch 1/20
157/157 [==============================] - 125s 760ms/step - loss: 0.6927 - accuracy: 0.5216 - val_loss: 0.6952 - val_accuracy: 0.4928
Epoch 2/20
157/157 [==============================] - 117s 746ms/step - loss: 0.6907 - accuracy: 0.5192 - val_loss: 0.6897 - val_accuracy: 0.5264
Epoch 3/20
157/157 [==============================] - 119s 756ms/step - loss: 0.6881 - accuracy: 0.5112 - val_loss: 0.6928 - val_accuracy: 0.5280
Epoch 4/20
157/157 [==============================] - 116s 740ms/step - loss: 0.6849 - accuracy: 0.5210 - val_loss: 0.6920 - val_accuracy: 0.5296
Epoch 5/20
157/157 [==============================] - 123s 785ms/step - loss: 0.6812 - accuracy: 0.5312 - val_loss: 0.6795 - val_accuracy: 0.5512
Epoch 6/20
157/157 [==============================] - 115s 733ms/step - loss: 0.6829 - accuracy: 0.5360 - val_loss: 0.6797 - val_accuracy: 0.5440
Epoch 7/20
157/157 [==============================] - 115s 733ms/step - loss: 0.6652 - accuracy: 0.5810 - val_loss: 0.6676 -

# **Saving and LSTM Network**

In [15]:
model_structure = model.to_json()
with open("lstm_model1.json", "w") as json_file:
  json_file.write(model_structure)
model.save_weights("lstm_weights1.h5")

In [16]:
from keras.models import model_from_json
with open("lstm_model1.json", "r") as json_file:
  json_string = json_file.read()
model = model_from_json(json_string)
model.load_weights('lstm_weights1.h5')

# **Prediction**

In [17]:
sample_1 = "I'm hate that the dismal weather that had me down for so long, when will it break! Ugh, when does happiness return?  The sun is blinding and the puffy clouds are too thin.  I can't wait for the weekend."

# We pass a dummy value in the first element of the tuple just because our helper expects it from the way processed the initial data.  That value won't ever see the network, so it can be whatever.
vec_list = tokenize_and_vectorize([(1, sample_1)])

# Tokenize returns a list of the data (length 1 here)
test_vec_list = pad_trunc(vec_list, maxlen)

test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))

print("Sample's sentiment, 1 - pos, 2 - neg : {}".format(model.predict(test_vec)))
print("Raw output of sigmoid function: {}".format(model.predict(test_vec)))

1/1 [==============================] - 1s 1s/step
Sample's sentiment, 1 - pos, 2 - neg : [[0.26347402]]
1/1 [==============================] - 0s 70ms/step
Raw output of sigmoid function: [[0.26347402]]


**Time and Performance Analysis**
When 2-3 layers are stacked in an LSTM (Long Short-Term Memory) network, the time and performance of the network can be affected in several ways:

Increased training time: Stacking LSTM layers increases the number of parameters that need to be learned during training. This can result in longer training times, especially for large datasets.

Improved performance: Adding more layers to an LSTM network can improve its ability to learn complex patterns in the data. This can result in better performance, especially on tasks that require long-term memory and complex sequences.